In [180]:
import config
import json
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pytesseract
from PIL import Image
import datetime
import time
from apscheduler.schedulers.tornado import TornadoScheduler
import math


In [197]:
with open('config.py', 'r', encoding='utf-8') as f:
    config = json.load(f)
    print(config)

{'login_url': 'https://scr.cyc.org.tw/tp10.aspx?Module=login_page&files=login', 'ID': 'E124262687', 'Password': 'a94003810912', 'active_day': [1, 3, 5]}


In [95]:
def init_driver(config):
    driver = webdriver.Chrome('./chromedriver')
    driver.get(config['login_url'])
    WebDriverWait(driver, 10).until(EC.alert_is_present())
    driver.switch_to.alert.accept()
    driver.switch_to.alert.accept()
    return driver

In [88]:
def get_captcha_image(driver):
    img = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_CaptchaImage"]')
    time.sleep(1)
    location = img.location
    size = img.size
    # left = location['x']
    # top = location['y']
    # right = left + size['width']
    # bottom = top + size['height']
    left = 2 * location['x']
    top = 2 * location['y']
    right = left + 2 * size['width'] - 10
    bottom = top + 2 * size['height']
    driver.save_screenshot('valicode.png')
    page_snap_obj = Image.open('valicode.png')
    image_obj = page_snap_obj.crop((left, top, right, bottom))
    image_obj.show()
    return image_obj

In [89]:
def image_handler(image_obj):
    img = image_obj.convert("L")  # 轉灰度圖
    pixdata = img.load()
    w, h = img.size
    threshold = 100
    # 遍歷所有畫素，大於閾值的為黑色
    for y in range(h):
        for x in range(w):
            if pixdata[x, y] < threshold:
                pixdata[x, y] = 0
            else:
                pixdata[x, y] = 255
    data = img.getdata()
    w, h = img.size
    black_point = 0
    for x in range(1, w - 1):
        for y in range(1, h - 1):
            mid_pixel = data[w * y + x]
            if mid_pixel < 50:
                top_pixel = data[w * (y - 1) + x]
                left_pixel = data[w * y + (x - 1)]
                down_pixel = data[w * (y + 1) + x]
                right_pixel = data[w * y + (x + 1)]
                if top_pixel < 10:
                    black_point += 1
                if left_pixel < 10:
                    black_point += 1
                if down_pixel < 10:
                    black_point += 1
                if right_pixel < 10:
                    black_point += 1
                if black_point < 1:
                    img.putpixel((x, y), 255)
                black_point = 0
    img.show()
    return img

In [90]:
def trans_img_2_string(img):
    result = pytesseract.image_to_string(img)
    # 可能存在異常符號，用正則提取其中的數字
    print(result)
    regex = '\d+'
    result = ''.join(re.findall(regex, result))
    print(result)
    return result

In [105]:

'''
使用者登入
'''
def user_login(driver, user_id, user_password, captcha_ans):
    account = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_loginid"]')
    account.clear()
    account.send_keys(user_id)
    password = driver.find_element_by_xpath('//*[@id="loginpw"]')
    password.clear()
    password.send_keys(user_password)
    captcha = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_Captcha_text"]')
    captcha.clear()
    captcha.send_keys(captcha_ans)
    driver.find_element_by_xpath('//*[@id="login_but"]').click()

In [199]:
def get_week_day(date):
    week_day = date.weekday()
    return week_day

In [200]:
get_week_day(datetime.datetime.today())

1

In [137]:
def get_QPid(base_url, field_id):
    trans_obj = {
        "1": 1133,
        "2": 1134,
        "3": 1135,
        "4": 1136,
        "5": 1137,
        "6": 1138,
    }
    target_QPid = base_url + '&QPid={}'.format(trans_obj[field_id])
    return target_QPid


'https://scr.cyc.org.tw/tp10.aspx?module=net_booking&files=booking_place&StepFlag=25&QPid=1136'

In [132]:
def get_QTime(base_url, time):
    target_QTime = base_url + '&QTime={}'.format(time)
    return target_QTime

In [133]:
def get_Date(base_url):
    today = datetime.datetime.today()
    target_day = today + datetime.timedelta(days=14)
    target_day_string = datetime.datetime.strftime(target_day, '%Y/%m/%d')
    print(target_day_string)
    target_Date = base_url + '&D={}'.format(target_day_string)
    return target_Date

2021/11/26


'https://scr.cyc.org.tw/tp10.aspx?module=net_booking&files=booking_place&StepFlag=25&QPid=1136&QTime=22&D=2021/11/26'

In [141]:
def get_field(base_url, QPid, time):
    base_url = get_QPid(base_url, QPid)
    base_url = get_QTime(base_url, time)
    base_url = get_Date(base_url)
    return base_url

In [65]:
url = 'https://scr.cyc.org.tw/tp10.aspx?module=net_booking&files=booking_place&StepFlag=25&QPid=1136&QTime=6&PT=1&D=2021/11/24'
driver.get(url)


In [ ]:
def field_handler(week_day):
    field_url = []
    if week_day == 1:
        
    elif week_day == 3:
    elif week_day == 5:
    return field_url


In [186]:
def main():
    try:
        with open('config.py', 'r', encoding='utf-8') as f:
            config = json.load(f)
            print(config)
        week_day = get_week_day(datetime.datetime.today())
        if week_day in config["active_day"]:
            global driver
            driver = init_driver(config)
            captcha_img = get_captcha_image(driver)
            captcha_img_gray = image_handler(captcha_img)
            captcha_ans = trans_img_2_string(captcha_img_gray)
            user_login(driver, config["ID"], config["Password"], captcha_ans)
            global url_list
            url_list = []
            base_url = "https://scr.cyc.org.tw/tp10.aspx?module=net_booking&files=booking_place&StepFlag=25"
            # 判斷星期幾要搶不同時段, 例如：週間搶20-22, 週末搶12-18
            field_url = field_handler(base_url, week_day)
            url_list.append(get_field(base_url, "4", 20))
            url_list.append(get_field(base_url, "4", 21))
            print(url_list)
            for t in range(len(url_list)):
                driver.execute_script("window.open('');")
            tomorrow_noon = datetime.datetime.combine(datetime.datetime.now().date(), datetime.time(0,0)) + datetime.timedelta(1)
            diff_time = tomorrow_noon - datetime.datetime.now()
            time.sleep(round(diff_time.total_seconds()))
            for i in range(len(url_list)):
                driver.switch_to.window(driver.window_handles[i+1])
                driver.get(url_list[i])
            time.sleep(5)
            driver.close()
        except Exception as error:
            print(error)




In [187]:
scheduler = TornadoScheduler()
scheduler.add_job(main, "cron", hour=23, minute=57, second=0)
# scheduler.add_job(main, "cron", hour=0, minute=0, second=0)
scheduler.start()


{'login_url': 'https://scr.cyc.org.tw/tp10.aspx?Module=login_page&files=login', 'ID': 'E124262687', 'Password': 'a94003810912'}


<ipython-input-95-69d0cb5138e1>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')
<ipython-input-88-51868adac0b3>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  img = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_CaptchaImage"]')


56080

56080


<ipython-input-105-155c8b45d397>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  account = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_loginid"]')
<ipython-input-105-155c8b45d397>:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_xpath('//*[@id="loginpw"]')
<ipython-input-105-155c8b45d397>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  captcha = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_Captcha_text"]')
<ipython-input-105-155c8b45d397>:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="login_but"]').click()


2021/11/26
2021/11/26
['https://scr.cyc.org.tw/tp10.aspx?module=net_booking&files=booking_place&StepFlag=25&QPid=1136&QTime=20&D=2021/11/26', 'https://scr.cyc.org.tw/tp10.aspx?module=net_booking&files=booking_place&StepFlag=25&QPid=1136&QTime=21&D=2021/11/26']


In [154]:
main()

{'login_url': 'https://scr.cyc.org.tw/tp10.aspx?Module=login_page&files=login', 'ID': 'E124262687', 'Password': 'a94003810912'}


<ipython-input-95-69d0cb5138e1>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')
<ipython-input-88-51868adac0b3>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  img = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_CaptchaImage"]')


64203

64203


<ipython-input-105-155c8b45d397>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  account = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_loginid"]')
<ipython-input-105-155c8b45d397>:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_xpath('//*[@id="loginpw"]')
<ipython-input-105-155c8b45d397>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  captcha = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_Captcha_text"]')
<ipython-input-105-155c8b45d397>:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="login_but"]').click()


2021/11/26
2021/11/26
['https://scr.cyc.org.tw/tp10.aspx?module=net_booking&files=booking_place&StepFlag=25&QPid=1136&QTime=20&D=2021/11/26', 'https://scr.cyc.org.tw/tp10.aspx?module=net_booking&files=booking_place&StepFlag=25&QPid=1136&QTime=21&D=2021/11/26']


In [179]:
tomorrow_noon = datetime.datetime.strptime("2021-11-12 16:10:00", "%Y-%m-%d %H:%M:%S")
diff_time = tomorrow_noon - datetime.datetime.now()
round(diff_time.total_seconds())


-226.131343